In [36]:
from collections import defaultdict
import copy
import random
import os
import glob
import shutil
import numpy as np

import matplotlib.pyplot as plt
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader, Dataset

In [37]:
cat_directory = "D:Dataset/dataset/etc/dogs_cats/cat/"
dog_directory = "D:Dataset/dataset/etc/dogs_cats/dog/"

cat_images_filepaths = sorted(glob.glob(cat_directory+"*"))
dog_images_filepaths = sorted(glob.glob(dog_directory+"*"))
print(len(cat_images_filepaths), len(dog_images_filepaths))
images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
label = [*[1.0]*12500,*[0.0]*12500]
dataset = list(zip(images_filepaths, label))

12500 12500


In [65]:
params = {
    "model": "resnet50",
    "device": "cuda:0",
    "lr": 0.001,
    "batch_size": 16,
    "num_workers": 0,
    "epochs": 10,
}

In [38]:
random.shuffle(dataset)
train_ds = dataset[:20000]
val_ds = dataset[20000:-100]
test_ds = dataset[-100:]
print(len(train_images_filepaths), len(val_images_filepaths), len(test_images_filepaths))

20000 4900 100


In [39]:
train_ds[:10]

[('D:Dataset/dataset/etc/dogs_cats/cat\\cat.11203.jpg', 1.0),
 ('D:Dataset/dataset/etc/dogs_cats/dog\\dog.4900.jpg', 0.0),
 ('D:Dataset/dataset/etc/dogs_cats/cat\\cat.3244.jpg', 1.0),
 ('D:Dataset/dataset/etc/dogs_cats/cat\\cat.4514.jpg', 1.0),
 ('D:Dataset/dataset/etc/dogs_cats/cat\\cat.4893.jpg', 1.0),
 ('D:Dataset/dataset/etc/dogs_cats/dog\\dog.10065.jpg', 0.0),
 ('D:Dataset/dataset/etc/dogs_cats/dog\\dog.1199.jpg', 0.0),
 ('D:Dataset/dataset/etc/dogs_cats/dog\\dog.6925.jpg', 0.0),
 ('D:Dataset/dataset/etc/dogs_cats/dog\\dog.2106.jpg', 0.0),
 ('D:Dataset/dataset/etc/dogs_cats/cat\\cat.11733.jpg', 1.0)]

In [60]:
class CatsVsDogsDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        (image_filepath, label) = self.dataset[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [63]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
train_dataset = CatsVsDogsDataset(dataset=train_ds, transform=train_transform)

val_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
val_dataset = CatsVsDogsDataset(dataset=val_ds, transform=val_transform)
test_dataset = CatsVsDogsDataset(dataset=test_ds, transform=val_transform)

In [67]:
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)
val_loader = DataLoader(
    val_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)
test_loader = DataLoader(
    test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

In [68]:
for img, label in train_loader:
    print(img.shape)
    break

torch.Size([16, 3, 128, 128])


In [72]:
import torchvision.models as models
import torch.nn as nn
import torch

model = getattr(models, params["model"])(pretrained=False, num_classes=1,)
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])